In [4]:
from modules.lib import parse_genome_property_file

In [14]:
global_gen_prop_dir = './data'
individual_gen_prop_dirs = [directory for directory in os.listdir(global_gen_prop_dir) if 'GenProp' in directory]

In [1]:
global_gen_prop_dir = './data/genomeProperties.txt'

In [5]:
parse_genome_property_file(global_gen_prop_dir)

ValueError: invalid literal for int() with base 10: 'IPP'